In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

from numpy import logical_and as npand
from numpy import logical_or as npor
import matplotlib

In [ ]:
from sympy import (symbols, pi, I, E, cos, sin, exp, tan, simplify, expand, factor, collect,
                   apart, cancel, expand_trig, diff, Derivative, Function, integrate, limit,
                   series, Eq, solve, dsolve, Matrix, N, preorder_traversal, Float, solve_linear_system,
                   eye, zeros, lambdify, Symbol,hessian, sqrt)
from sympy.physics.mechanics import dynamicsymbols, init_vprinting

In [ ]:
init_vprinting()

In [ ]:
from sympy.physics.mechanics import ReferenceFrame, Point, LagrangesMethod, Lagrangian, inertia, RigidBody, dot

In [ ]:
N = ReferenceFrame('N')
origen = Point('origen')
origen.set_vel(N, 0)


In [ ]:
t = symbols('t')
t

In [ ]:
x, y, psi = dynamicsymbols('x y psi')
x, y, psi

In [ ]:
fi0, fi1, fi2, fi3 = dynamicsymbols('phi:4')
fi0, fi1, fi2, fi3

In [ ]:
CM = origen.locatenew('CM', x*N.x + y*N.y)
CM.set_vel(N, CM.pos_from(origen).dt(N))

In [ ]:
B = N.orientnew('B', 'Axis', (psi, N.z))

In [ ]:
m_r, m_w, i_r, i_w, l0, l1, r = symbols('m_r m_w i_r i_w l0 l1 r')

In [ ]:
I_r = inertia(B, 0, 0, i_r)
body = RigidBody('Body', CM, B, m_r, (I_r,CM))

In [ ]:
wheel_params = [
    [45,  l0,  l1, fi0],
    [-45, l0, -l1, fi1],
    [-45, -l0, l1, fi2],
    [45, -l0, -l1, fi3]
]

In [ ]:
wheel_axes = []
wheel_centers = []
I_w = inertia(B, 0, i_w, 0)
wheel_bodies = []
for ii in range(4):
    wheel_axis = B.orientnew(f'W{ii}', 'Axis', (wheel_params[ii][3], B.y))
    wheel_axes.append(wheel_axis)
    wheel_center = CM.locatenew(f'CMW{ii}', wheel_params[ii][1]*B.x + wheel_params[ii][2]*B.y)
    wheel_center.set_vel(N, wheel_center.v2pt_theory(CM, N, B))
    wheel_centers.append(wheel_center)
    wheel_body = RigidBody(f'Wheel{ii}', wheel_center, wheel_axis, 0, (I_w,wheel_center))
    wheel_bodies.append(wheel_body)
wheel_axes, wheel_centers, wheel_bodies

In [ ]:
lag_f = simplify(Lagrangian(N, body, *wheel_bodies))
lag_f

## Restrictions:

In [ ]:
from optibot.symbolic import integerize

In [ ]:
non_hol_rest = []
for ii in range(4):
    angle = np.radians(wheel_params[ii][0])
    aux_syst = B.orientnew('aux', 'Axis', (angle, B.z))
    contact_p = wheel_centers[ii].locatenew('contact_p', -r*N.z)
    contact_p.set_vel(N, contact_p.v2pt_theory(wheel_centers[ii], N, wheel_axes[ii]))
    raw_restr = dot(contact_p.vel(N),aux_syst.y)
    non_hol_rest.append(simplify(integerize((2/2**0.5)* raw_restr)))

In [ ]:
Matrix(non_hol_rest)

## Lagrange system

In [ ]:
FL = []
for ii in range(4):
    u = symbols(f'u{ii}')
    FL.append((wheel_axes[ii], u*B.y))

In [ ]:
from optibot.symbolic import SimpLagrangesMethod

In [ ]:
LM_simp = SimpLagrangesMethod(lag_f, [x, y, psi, fi0, fi1, fi2, fi3] ,
                              forcelist=FL, frame=N, nonhol_coneqs=non_hol_rest)

In [ ]:
LM_simp.Q

In [ ]:
LM_simp.M

In [ ]:
LM_simp.phi_q

In [ ]:
LM_simp.H

In [ ]:
LM_simp.K

In [ ]:
LM_simp.Fa

In [ ]:
LM_simp.RHS_reduced

In [ ]:
LM_simp.calculate_RHS()

In [ ]:
LM_simp.calculate_RHS_full()

### Reference: based on Lagranges Method

In [ ]:
LM = LagrangesMethod(lag_f, [x, y, psi, fi0, fi1, fi2, fi3] , forcelist=FL, frame=N, nonhol_coneqs=non_hol_rest)

In [ ]:
LM.form_lagranges_equations()

### Reference: also works without constraints

In [ ]:
LM_simp_no_const = SimpLagrangesMethod(lag_f, [x, y, psi, fi0, fi1, fi2, fi3] ,
                              forcelist=FL, frame=N, nonhol_coneqs=None)

In [ ]:
LM_simp_no_const.calculate_RHS_full()